In [17]:
from decimal import Decimal, getcontext
import numpy as np
from fractions import Fraction
import sympy as sm
import pandas as pd

eps0 = Fraction(8.85e-12)
e = Fraction(1.6e-19)
p = 1e5
NE0 = 1e18
E0 = 5e3
F0 = 1500
U1 = -100
U2 = 0
mue = 1
mui = 1
alpha = 1
ne_0 = 1e18

N = 50
M = N

L = 1
R = 1
r1 = R
hr = Fraction(1/(N-0.5))
hz = Fraction(L/(M-1))
tEnd = 1
tau = 1
 
r = sm.zeros(1,N)
for i in range(N):
    r[i] = Fraction(hr/2 + (R-hr/2)/(N-1)*(i)) #r = np.linspace(hr/2,R,N)

z = sm.zeros(1,M)
for i in range(M):
    z[i] = Fraction(0 + L/(M-1)*(i)) #z = np.linspace(0,L,M)

FF = sm.zeros(N,M)
for j in range(0,M):
    for i in range(0,N):
        FF[i,j] = Fraction(-50*z[j]**2)
for i in range(N):
    FF[i,0] = 0 #низ
for i in range(N):
    FF[i,M-1] = -50

In [18]:
def fsolve(FF,N,M,r,hr,hz,omega_relax =1.8):
    FF1 = FF.copy()
    eps0 = Fraction(8.85e-12) #электрическая постоянная
    e = Fraction(1.6e-19) #заряд электрона
    f = sm.ones(N,M) * (100)
    d = sm.ones(1,N*M) #формируем диагональ матрицы СЛАУ
    l1 = sm.zeros(1,N*M) 
    l2 = sm.zeros(1,N*M)
    for j in range(1,M-1):
        for i in range(1,N-1):
            k = i+j*N
            d[k] = 2/hr**2 +2/hz**2
            l1[k] = -(r[i]-hr/2)/(r[i]*hr**2)
            l2[k] = -1/hz**2
        l1[int((j+1)*N-1)] = -1/hr**2

    tol = Fraction(1e-2)
    rn_finish = Fraction(1e32)

    kiter = 0
    N_iter = 5

    for p in range(N_iter):
        norm_dif = 0
        rn = 0.
        print('rn = ',rn)

        print("Расчет")
      
        resid = sm.zeros(1,N*M)
        corrector = sm.zeros(1,N*M)


        for j in range(1,M-1):
            # Граничные условия второго рода слева  
            rr = (FF[0,j]*(2/hr**2) - FF[1,j]*(2/hr**2))\
                - 1/hz**2 * FF[0,j-1] + 2/hz**2 * FF[0,j] - 1/hz**2 * FF[0,j+1]\
                - f[0,j]
            resid[j*N] = rr
            rn = rn+rr**2
            print('rr невязка левые граница', rr )
            FF1[0,j] = FF[0,j] - (omega_relax*rr)/d[j*N]
            norm_dif = max(norm_dif,abs((omega_relax*rr)/(d[j*N])))
            
            # Обходим внутренние узлы
            for i in range(1,N-1):
                A1 = -(FF[i-1,j]*(r[i]- hr/2) -2*FF[i,j]*r[i] + FF[i+1,j]*(r[i]+hr/2)) / (hr**2 * r[i])
                A2 = -(FF[i,j-1] - 2*FF[i,j] + FF[i,j+1])/hz**2
                rr = A1  + A2 - f[i,j]
                corrector[i+j*N] = rr - omega_relax*(l1[i+j*N-1]*corrector[i+j*N -1] 
                                                     - l2[i+(j-1)*N]*corrector[i+(j-1)*N])/d[i+j*N]
              
                resid[i+j*N] = rr
                rn = rn+rr**2
                print ('i = ',i,'j = ', j, 'rr = ',rr,'rn = ',rn)
                FF1[i,j] = FF[i,j] - omega_relax*corrector[i+j*N]
                norm_dif = max(norm_dif,abs(omega_relax*corrector[i+j*N]))

            # справа
            rr = (FF[N-1,j] - FF[N-2,j])*2 / hr**2 - f[N-1,j]\
                - 1/hz**2 * (FF[N-1,j-1] - 2*FF[N-1,j] + FF[N-1,j+1])
            corrector[(j+1)*N-1] = rr - omega_relax*(l1[(j+1)*N-2]*corrector[(j+1)*N-2] 
                                                    - l2[j*N-1]*corrector[j*N-1])/d[(j+1)*N-1]
            print('невязка правой границы',rr,'\n')
            resid[(j+1)*N-1] = rr
            rn = rn+rr**2
            FF1[N-1,j] = FF[N-1,j] - (omega_relax*corrector[(j+1)*N-1])
            norm_dif = max(norm_dif,abs(omega_relax*corrector[(j+1)*N-1]))
      
        ddd=np.max(np.abs(FF-FF1))
        try:
            err_c = norm_dif/ddd
        except:
            err_c = 0
        FF=FF1.copy()
        rn = rn*hr*hz
        kiter = p
        rn_finish = rn
        #print("итерация номер ", k, " невязка = ", rn," погрешность = ", err_c)
        if rn_finish < tol**2:
            break
      
    print("число итераций =  ", kiter+1)
    print("невязка приближенного решения",rn)
    print("относительная разность, norm_dif = ", norm_dif)
    return FF,rn

In [19]:
print('решение')
t = 0
while t<= tEnd:
    
    FF,nv = fsolve(FF,N,M,r,hr,hz)
    t = tau+t

print((FF.transpose()))
for i in range(N): 
    for j in range(N): 
        print(float(FF[i,j]), end = ' ')
    print()

решение
rn =  0.0
Расчет
rr невязка левые граница 0
i =  1 j =  1 rr =  0 rn =  0
i =  2 j =  1 rr =  0 rn =  0
i =  3 j =  1 rr =  0 rn =  0
i =  4 j =  1 rr =  0 rn =  0
i =  5 j =  1 rr =  0 rn =  0
i =  6 j =  1 rr =  0 rn =  0
i =  7 j =  1 rr =  0 rn =  0
i =  8 j =  1 rr =  0 rn =  0
i =  9 j =  1 rr =  0 rn =  0
i =  10 j =  1 rr =  0 rn =  0
i =  11 j =  1 rr =  0 rn =  0
i =  12 j =  1 rr =  0 rn =  0
i =  13 j =  1 rr =  0 rn =  0
i =  14 j =  1 rr =  0 rn =  0
i =  15 j =  1 rr =  0 rn =  0
i =  16 j =  1 rr =  0 rn =  0
i =  17 j =  1 rr =  0 rn =  0
i =  18 j =  1 rr =  0 rn =  0
i =  19 j =  1 rr =  0 rn =  0
i =  20 j =  1 rr =  0 rn =  0
i =  21 j =  1 rr =  0 rn =  0
i =  22 j =  1 rr =  0 rn =  0
i =  23 j =  1 rr =  0 rn =  0
i =  24 j =  1 rr =  0 rn =  0
i =  25 j =  1 rr =  0 rn =  0
i =  26 j =  1 rr =  0 rn =  0
i =  27 j =  1 rr =  0 rn =  0
i =  28 j =  1 rr =  0 rn =  0
i =  29 j =  1 rr =  0 rn =  0
i =  30 j =  1 rr =  0 rn =  0
i =  31 j =  1 rr =  0 rn =

In [20]:
Matrix=FF.copy()
for i in range(N): 
    for j in range(N): 
        print(float(Matrix[i,j]), end = ' ')
    print()

print()
FFF = sm.zeros(N,M)
for j in range(0,M):
    for i in range(0,N):
        FFF[i,j] = Fraction(-50*z[j]**2)
for i in range(N):
    FFF[i,0] = 0 #низ
for i in range(N):
    FFF[i,M-1] = -50
for i in range(N): 
    for j in range(N): 
        print(float(FFF[i,j]), end = ' ')
    print()

print()
for i in range(N): 
    for j in range(N): 
        print(float(Matrix[i,j]-FFF[i,j]), end = ' ')
    print()


0.0 -0.02082465678990815 -0.08329862398591759 -0.18742190889692245 -0.3331945058173531 -0.5206164006843051 -0.7496876360404066 -1.0204081644144822 -1.3327780077155993 -1.686797168733972 -2.0824656518586395 -2.519783373103251 -2.9987506001534245 -3.519366848846265 -4.081632734103383 -4.685547641332039 -5.331111980047782 -6.018325829511435 -6.747188543191845 -7.517701073595093 -8.329862444901098 -9.183673583541552 -10.079133627845426 -11.016243203632692 -11.995002003171953 -13.015410556150654 -14.077467505423424 -15.181174471710634 -16.32653054803209 -17.51353637359811 -18.742190508474973 -20.01249474670122 -21.32444822053701 -22.678050629397248 -24.07330324431088 -25.510203772649174 -26.988754640416804 -28.50895451966155 -30.070804284190817 -31.6743020528906 -33.319450168250455 -35.00624749399128 -36.73469364644666 -38.50479024356892 -40.31653438659582 -42.16992914000993 -44.064973032328 -46.00166592439451 -47.980008347779304 -50.0 
0.0 -0.02082465678990815 -0.08329862379620326 -0.18742